In [111]:
import sys
# !{sys.executable} -m pip install python-igraph simpy numpy scipy matplotlib
# !{sys.executable} -m pip install cpython
# !{sys.executable} -m pip install runstats
# !{sys.executable} -m pip install pandas

In [102]:
import igraph
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng, RandomState
import pandas as pd
from scipy import stats
from simpy import *
from simpy.events import AnyOf, AllOf, Event
from simpy.resources.store import *

# Preliminaries

## Graphs

Chainweb graphs are undirected and regular graphs that are choosen to have low degree and low diameter. 

Each vertex of a Chainweb graph represents a chain and edges designate the dependencies between blocks on different chains. A block on chain $c$ at height $h$ has to reference each block of height $h-1$ on all chains that are adjacent to $c$. In addition a block at height $h$ also depends on the block at height $h-1$ on the same chain. We also say that a chain $c$ depends on a chan $d$ if $c$ is adjacent to $d$.

Since Chainweb graphs are undirected, dependencies between chains are symmetric. Thus, In the following we encode dependencies using directed, symmetric directed graphs. We also make them reflexiv, by making each vertex adjacent to itself, because each each block depends on its parent on the same chain.

The $\text{Peterson}$ graph is of minimal order (number of vertices) for degree 3 and diameter 2. 

In [5]:
def peterson():
    graph = igraph.Graph(directed=True)
    graph.add_vertices(10)
    graph.add_edges(
        [ (0, 0), (0, 2), (0, 3), (0, 5)
        , (1, 1), (1, 3), (1, 4), (1, 6)
        , (2, 2), (2, 0), (2, 4), (2, 7)
        , (3, 3), (3, 0), (3, 1), (3, 8)
        , (4, 4), (4, 1), (4, 2), (4, 9)
        , (5, 5), (5, 0), (5, 6), (5, 9)
        , (6, 6), (6, 1), (6, 5), (6, 7)
        , (7, 7), (7, 2), (7, 6), (7, 8)
        , (8, 8), (8, 3), (8, 7), (8, 9)
        , (9, 9), (9, 4), (9, 5), (9, 8)
        ]
    )
    return(graph)

Peterson = peterson()

The $\text{Twenty}$ chain graph is of minimal order for degree 3 and diameter 3.

In [6]:

def twenty():
    graph = igraph.Graph(directed=True)
    graph.add_vertices(20)
    graph.add_edges(
        [ (0, 0), (0, 10), (0,15), (0,5)
        , (1, 1), (1, 11), (1,16), (1,6)
        , (2, 2), (2, 12), (2,17), (2,7)
        , (3, 3), (3, 13), (3,18), (3,8)
        , (4, 4), (4, 14), (4,19), (4,9)
        , (5, 5), (5, 0), (5,7), (5,8)
        , (6, 6), (6, 1), (6,8), (6,9)
        , (7, 7), (7, 2), (7,5), (7,9)
        , (8, 8), (8, 3), (8,5), (8,6)
        , (9, 9), (9, 4), (9,6), (9,7)
        , (10, 10), (10, 0), (10,11), (10,19)
        , (11, 11), (11, 1), (11,10), (11,12)
        , (12, 12), (12, 11), (12,13), (12,2)
        , (13, 13), (13, 12), (13,14), (13,3)
        , (14, 14), (14, 13), (14,15), (14,4)
        , (15, 15), (15, 0), (15,14), (15,16)
        , (16, 16), (16, 1), (16,15), (16,17)
        , (17, 17), (17, 16), (17,18), (17,2)
        , (18, 18), (18, 17), (18,19), (18,3)
        , (19, 19), (19, 10), (19,18), (19,4)
        ]
    )
    return (graph)

Twenty = twenty()

## Histogram

Quick and Dirty Linear Histograms.

In [7]:
# TODO: there are probably some good libraries with support for this
#
class Histogram:

    def __init__(self, binSize):
        self.bins = {}
        self.binSize = binSize

    def sample(self, x):
        i = math.floor(x / self.binSize)
        if i in self.bins:
            self.bins[i] += 1
        else:
            self.bins[i] = 1

    def append(self, h):
        if h.binSize == self.binSize:
            for i in h.bins:
                if i in self.bins:
                    self.bins[i] += h.bins[i]
                else:
                    self.bins[i] = h.bins[i]
        else:
            throw("merging histograms with different bin sizes isn't yet implemented")

    def result(self):
        imin = min(self.bins.keys())
        imax = max(self.bins.keys())
        a = [0] * (imax - imin + 1)
        b = [self.binSize * k for k in range(imin, imax + 1)]
        for i in range(imin, imax + 1):
            if i in self.bins:
                a[i - imin] = self.bins[i]
        return a, b

# TODO: implement more general WithStatistics class
class HistogramSample:
    def __init__(self, histogram, clock):
        self.histogram = histogram
        self.clock = clock
    def __enter__(self):
        self.start = self.clock.now
        return self.start
    def __exit__(self, type, value, traceback):
        self.histogram.sample(self.clock.now - self.start)
        return None # rethrow

# Simulation Model

The goal of the simulation is to investigate the interaction of dependencies between chains and difficulty adjustement in the computation of a single chain. 

Block production (*mining*) on each chain is exponentially distributed.
The mean mining time is called the $\text{target}$. Mining on a chain can only proceed if all dependencies are available. I.e. a block on chain $c$ at height $h$ can be mined only after all blocks at height $h-1$ have been mined on $c$ and all chains adjacent to $c$. We say that a chain is *blocked* when it is done mining a block and while it is waiting for dependencies for the next block to become available. The overall *block time* is the time that it takes a chain to do a full cycle from starting to mine a block to starting to mine the next block, i.e. the time that mining takes, plus the time it is blocked, plus any potential additional latencies. 

POW blockchain algorithms are desigined such that the overall block time resembles an exponential distriution with a particular mean. The goal of *Difficulty Adjustment* is to adjust the target such that the observed mean matches the targeted mean. Single chain POW block chains can't be blocked and block times are exponentially distributed when the mining time dominates the other latencies. This can be achieved by choosing the a block time that is sufficiently large compared to network propgation and validation times. (For the purpose of this research, we are ignoring races that result in orphans.) In Chainweb chains can additionally be blocked and the distribution of the block time results from the the combination of the mining distribution, the distribution of times that chains are blocked, and the distribution of other latencies (network propagation and block validation).

The model is designed to investigate how the choice of the difficulty adjustment algorithm affect the overall resulting block time distribution in Chainweb.

We don't model consensus. Therefore the model doesn't include races between distributed nodes and orphan blocks. Because there are no conflicts only a single block for each chain and height is produced. Thus, blocks are uniquly identified by their chain and height. Also, each block is used exactly once by each adjacent chain. In other words a single linear chainweb is produced without any forks. 

Each vertex in the graph is labeled with a `Chain`, which is implemented as a `Process` that mines blocks on that chain. 

Dependencies between chain processes are implemented by passing messages with new blocks to adjacent chains. A chain can only produce a new block if all of its dependencies have been received. Each (directed) edge is assigned a communication channel. The source chain of an edge publishes new blocks to the channel. The target of the edge can receive a block ar specific height by awaiting that block on the channel.

## Configuration

In [8]:
class Config:
    # the argument is a Class, not an object!
    def __init__(self, da):
        self.DA = da
    
    # All durations are in seconds

    # targeted mean block time
    MEAN_BLOCK_TIME = 30

    # number of blocks in an epoch
    EPOCH = 120

    # Median delay of pact new block call delay
    PACT_VALIDATION_TIME = 0.01
    PACT_NEW_BLOCK_TIME = PACT_VALIDATION_TIME

    # The mean latency for propagating a block in the network
    NETWORK_LATENCY = 0.5

    # overall block latencies (validation + network + validation, including pact validation)
    NUMBER_OF_MINING_NODES = 3
    LOCAL_BLOCK_LATENCY = 2 * PACT_VALIDATION_TIME
    REMOTE_BLOCK_LATENCY = 2 * PACT_VALIDATION_TIME + NETWORK_LATENCY
    
    GRAPH = Twenty

    # Seed for the PRNG
    RNG_SEED = 17
    
    # Difficulty Adjustement
    BLOCK_TIME_CHOICE = 0.9 # how close to the solve time the block time is chosen (0,1)
    
    # Multiple Hash functions:
    NUMBER_OF_HASH_FUNCTIONS = 1
    MAX_TARGET_MULTIPLIER = 1

### Durations

We use the generic term $\text{latency}$ to summarize the duration of the validation of a block after it is discovered and before it is published, the network latency, and the validation after it is received. In case a block is discovered on the same node the network latency and one validation step are omitted. This is modeled by using a Bernoulli distribution to estimate the chance that a block is mined locally and a reduced latency is used.

Pact new block validation is modeled separately because, depending on the mining framework that is used, it is applied after a chain becomes unblocked.

In [9]:
def gamma5(prng, mean : float) -> float:
    return prng.gamma(shape=5, scale=mean/5)

# latency: validation + network + validation
def latencyDelay(ctx) -> float:
    is_local = ctx.prng.binomial(n=1, p = 1 / ctx.conf.NUMBER_OF_MINING_NODES)
    if is_local:
        return gamma5(ctx.prng, ctx.conf.LOCAL_BLOCK_LATENCY)
    else:
        return gamma5(ctx.prng, ctx.conf.REMOTE_BLOCK_LATENCY)

# Pact new block
def pactDelay(ctx) -> float:
    return gamma5(ctx.prng, ctx.conf.PACT_NEW_BLOCK_TIME)

# Mining is exponentialy distributed with the target being the mean
def miningDelay(ctx, target : float) -> float:
    return ctx.prng.exponential(target)

## Blocks

In [10]:
class Block:
    def __init__(self, chainId, height, time, epochTime, target):
        self.chainId = chainId
        self.height = height
        self.time = time

        # For Da
        self.epochTime = epochTime
        self.target = target

    def __lt__(self, other):
        return self.height < other.height

## Difficulty Adjustment Algorithms

In [11]:
def avg(a : list[float]) -> float: 
    return sum(a) / len(a)

# DA that does local adjustments based on a globally
# synchronized epoch.
#
# This is what is currently implemented on mainnet.
#
class ChainDA:
    def __init__(self, ctx):
        self.ctx = ctx
        self.env = self.ctx.env
        self.conf = self.ctx.conf
        
    def isEpochStart(self, parent : Block) -> bool:
        if parent == None:
            return True
        else:
            return (parent.height + 1) % self.conf.EPOCH == 0

    def run(self, parent : Block, parents : [Block]) -> (float, float):
        return self.runChain(parent)
    
    def runChain(self, parent : Block):
        if parent == None:
            return self.env.now, self.conf.MEAN_BLOCK_TIME
        if self.isEpochStart(parent):
            epochTime = parent.time
            target = parent.target * (self.conf.MEAN_BLOCK_TIME * self.conf.EPOCH) / (parent.time - parent.epochTime)
        else:
            epochTime = parent.epochTime
            target = parent.target
        return epochTime, target

# DA that is based on globally synchronized epoch. It performs
# chainDa and takes the average for the chain along with all it's
# adjacent chains.
#
class AvgDA(ChainDA):
    
    def __init__(self, env): super().__init__(env)

    def run(self, parent, parents):
        if parent == None:
            return self.runChain(parent)
        if self.isEpochStart(parent):
            ps = list(parents)
            ps.append(parent)
            epochTimes, targets = zip(*[self.runChain(p) for p in ps])
            epochTime = avg(epochTimes)
            target = avg(targets)
        else:
            epochTime = parent.epochTime
            target = parent.target
        return epochTime, target

  
# A DA algorithm that adjusts difficutly on a per chain basis
# based on the solve time for that chain.
#
class LocalDA(ChainDA):
    def __init__(self, env): super().__init__(env)

    def run(self, parent, parents):
        if parent == None:
            return self.runChain(parent)
        if self.isEpochStart(parent):
            return self.runChain(parent)
        else:
            ps = list(parents)
            ps.append(parent)
            penalty = avg([p.time for p in ps]) - parent.time # approximation of wait time
            epochTime = parent.epochTime + penalty
            target = parent.target
        return epochTime, target

## Mining

The target multiplier can be used to simulate a situation where different chains receive different
amounts of hash power or use different hash algorithms.

In [12]:
def targetMultiplier(conf : Config, chainId : int) -> float:
    n = conf.NUMBER_OF_HASH_FUNCTIONS
    m = conf.MAX_TARGET_MULTIPLIER
    return (1 + (chainId % n)) * (m / n)

class Mine(Process):
    def __init__(self, ctx, chainId, parent, parents):
        
        self.ctx = ctx
        self.env = ctx.env
        self.conf = ctx.conf
        self.da = ctx.da

        self.parent = parent
        self.parents = parents
        self.chainId = chainId
        self.targetMultiplier = targetMultiplier(self.conf, chainId)

        super().__init__(self.env, self.run())


    def run(self):
        # adjust difficulty
        epochTime, target = self.da.run(self.parent, self.parents)
        
        height = 0 if self.parent == None else self.parent.height + 1

        # mine
        solveTime = miningDelay(self.ctx, target * self.targetMultiplier)
        newTime = self.env.now + solveTime * self.conf.BLOCK_TIME_CHOICE # current miner behavior: 0
        yield self.env.timeout(solveTime)

        # create new block
        block = Block(
                chainId = self.chainId,
                height = height,
                time = newTime,
                epochTime = epochTime,
                target = target
            )
        return block

## Chain

In [13]:
class AwaitParents(Process):
    def __init__(self, env, graph, block : Block):
    
        def run():
            # No parents for the genesis block
            if block == None: return []
    
            chainId = block.chainId
            edges = graph.vs[chainId].in_edges()
            f = lambda b: b.height == block.height
            results = yield AllOf(env, [ FilterStoreGet(e["link"], f) for e in edges ])
            parents = list(results.values())
            return parents
    
        super().__init__(env, run())

In [14]:
# asynchronously publish block to each adjacent chain
#
# TODO should this an process or is it fine for it to be instantaneous?
#
class Publish:
    def __init__(self, ctx, block):
        self.ctx = ctx
        self.conf = ctx.conf
        self.env = ctx.env
        self.graph = ctx.conf.GRAPH
        self.block = block
        self.chain = block.chainId
        self.run()

    def run(self):
        for e in self.graph.vs[self.chain].out_edges():
            self.env.process(self.publishToChain(e))
        
    def publishToChain(self,e):
        yield self.env.timeout(latencyDelay(self.ctx))
        yield StorePut(e["link"], self.block)

In [15]:
class Chain:
    def __init__(self, ctx, chainId : int):

        # Context
        self.ctx = ctx
        self.conf = ctx.conf
        self.env = ctx.env
        self.graph = ctx.graph
        self.chain = chainId
        self.da = ctx.da

        # State
        self.currentBlock = None

        # Monitors and Statistics
        self.isBlocked = False        
        self.cycleTimes = []
        self.blockedTimes = []
        self.newBlockTimes = []
        self.mineTimes = []
        self.targets = []

        # start chain process
        self.action = self.env.process(self.run())
        
    def run(self):
        t=None
        while True:
            t0 = self.env.now

            # await parents for new block
            t1 = self.env.now
            self.isBlocked = True
            parents = yield AwaitParents(self.env, self.graph, self.currentBlock)
            self.isBlocked = False
            self.blockedTimes.append(self.env.now - t1)

            # pact new block (TODO: do we call this here? There should
            # no reason to wait for *all* adjacents for calling new block)
            d = pactDelay(self.ctx)
            yield self.env.timeout(d)
            self.newBlockTimes.append(d)

            # mine
            t1 = self.env.now
            block = yield Mine(self.ctx, self.chain, self.currentBlock, parents)
            self.mineTimes.append(self.env.now - t1)
            self.currentBlock = block
            self.targets.append(block.target)
                    
            # publish block asynchronously
            Publish(self.ctx, block)
                
            self.cycleTimes.append(self.env.now - t0)

## Chainweb

In [16]:
class Chainweb:

    def __init__(self, ctx):
        self.ctx = ctx
        
        self.graph = self.ctx.graph
        self.env = self.ctx.env
        self.da = self.ctx.da
        self.conf = self.ctx.conf

        for i in self.graph.es:
            i["link"] = FilterStore(self.env)

        for i in self.graph.vs:
            chain = Chain(self.ctx, i.index)
            i["chain"] = chain
        
        self.chains = self.graph.vs["chain"]

# Statistics

In [17]:
# Monitor the value of variables at fixed intervals
#
def monitor(env, chainweb, n):
    blocked = []
    while True:
        yield env.timeout(n)
        cur = [ i.isBlocked for i in chainweb.graph.vs["chain"] ]
        blocked.append(sum(cur)/ len(cur))
        print("[%d][MONITOR] %d%% blocked" % (env.now, 100 * sum(cur) / len(cur)))

# Main

In [18]:
class Ctx:
    def __init__(self, env, conf):
        self.env = env
        self.conf = conf
        self.graph = conf.GRAPH
        
        # Initialize DA object
        self.da = conf.DA(self)
        
        # PRNGs
        self.prng = default_rng(conf.RNG_SEED)
        self.prng_ = RandomState(conf.RNG_SEED) # scipy uses the legacy generator

In [19]:
def main(conf, n):
    env = Environment()
    ctx = Ctx(env = env, conf = conf)
    cw = Chainweb(ctx)
    m = env.process(monitor(env, cw, n/10))
    env.run(until=n)
    return cw

# Run Simulation

We run the simulation in three different configurations:

1.  No chain target multipliers and `ChainDA`
2.  No chain target mulitpliers and `AvgDA`
3.  per chain target multipliers and `LocalDA`

Different target multipliers simulate a situation where different chains receive different hash power or use different hash algorithms.

Results for scenario 1. and 2. don't depend on the choice of that block time within the allowed range, i.e. strictly larger than the parent time and smaller than the real time when the block is solved. But both
scenarios can't handle target multipliers.

Scenario 3. yields robust results when target multipliers are used, but depends on the block times chosen to be close to the actual solve time. If the block times that don't depend on the solve time result in unstable DA and very long blocked times on some chains, which means that mining on those chains becomes a lotery that doesn't depend on hash power.

In the third scenario it seems that the DA is unstable under poorly chosen block times even if no multipliers are used (TODO: double check that claim). This indicates that the natural noise in the solve times is amplified by the algorithm. The quality of the choice of the block time affects the convergence speed. It's not yet clear whether it also affects the quality of DA once the chains reach their plateau.

In [20]:
n = 500000

## LocalDA

Run standard config with LocalDA and

* 1 hash function
* pact validation 0.02sec
* block times ranging from 0.1 to 0.9 of solve time

In [21]:
conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.1
cw_localDA_01 = main(conf, n)

conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.2
cw_localDA_02 = main(conf, n)

conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.3
cw_localDA_03 = main(conf, n)

conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.4
cw_localDA_04 = main(conf, n)

conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.5
cw_localDA_05 = main(conf, n)

conf = Config(LocalDA)
conf.BLOCK_TIME_CHOICE = 0.9
cw_localDA_09 = main(Config(LocalDA), n)

[50000][MONITOR] 70% blocked
[100000][MONITOR] 75% blocked
[150000][MONITOR] 50% blocked
[200000][MONITOR] 60% blocked
[250000][MONITOR] 60% blocked
[300000][MONITOR] 60% blocked
[350000][MONITOR] 65% blocked
[400000][MONITOR] 70% blocked
[450000][MONITOR] 65% blocked
[50000][MONITOR] 65% blocked
[100000][MONITOR] 60% blocked
[150000][MONITOR] 80% blocked
[200000][MONITOR] 85% blocked
[250000][MONITOR] 65% blocked
[300000][MONITOR] 70% blocked
[350000][MONITOR] 70% blocked
[400000][MONITOR] 80% blocked
[450000][MONITOR] 90% blocked
[50000][MONITOR] 55% blocked
[100000][MONITOR] 50% blocked
[150000][MONITOR] 60% blocked
[200000][MONITOR] 70% blocked
[250000][MONITOR] 75% blocked
[300000][MONITOR] 60% blocked
[350000][MONITOR] 85% blocked
[400000][MONITOR] 65% blocked
[450000][MONITOR] 70% blocked
[50000][MONITOR] 35% blocked
[100000][MONITOR] 60% blocked
[150000][MONITOR] 70% blocked
[200000][MONITOR] 65% blocked
[250000][MONITOR] 70% blocked
[300000][MONITOR] 60% blocked
[350000][MONIT

## AvgDA

Run standard config with AvgDA and

* 1 hash function
* pact validation 0.02sec

In [22]:
cw_avgDA = main(Config(AvgDA), n)

[50000][MONITOR] 40% blocked
[100000][MONITOR] 45% blocked
[150000][MONITOR] 65% blocked
[200000][MONITOR] 80% blocked
[250000][MONITOR] 85% blocked
[300000][MONITOR] 55% blocked
[350000][MONITOR] 60% blocked
[400000][MONITOR] 60% blocked
[450000][MONITOR] 60% blocked


## ChainDA

Run standard config with ChainDA and

* 1 hash function
* pact validation 0.02sec

In [23]:
cw_chainDA = main(Config(ChainDA), n)

[50000][MONITOR] 60% blocked
[100000][MONITOR] 50% blocked
[150000][MONITOR] 65% blocked
[200000][MONITOR] 65% blocked
[250000][MONITOR] 60% blocked
[300000][MONITOR] 70% blocked
[350000][MONITOR] 70% blocked
[400000][MONITOR] 65% blocked
[450000][MONITOR] 65% blocked


# Process and Store Results

In [25]:
def getData(cw):
    chains = cw.chains
    cs = pd.DataFrame()
    cs["cycle times"] = pd.DataFrame(dict([(c.chain, pd.Series(c.cycleTimes)) for c in chains])).stack()
    cs["blocked times"] = pd.DataFrame(dict([(c.chain, pd.Series(c.blockedTimes)) for c in chains])).stack()
    cs["targets"] = pd.DataFrame(dict([(c.chain, pd.Series(c.targets)) for c in chains])).stack()
    cs["new block times"] = pd.DataFrame(dict([(c.chain, pd.Series(c.newBlockTimes)) for c in chains])).stack()
    cs["mine times"] = pd.DataFrame(dict([(c.chain, pd.Series(c.mineTimes)) for c in chains])).stack()
    cs.index.names = ["height", "chain"]
    return cs

In [27]:
data = pd.DataFrame({
    "local_DA_01": getData(cw_localDA_01).stack(),
    "local_DA_02": getData(cw_localDA_02).stack(),
    "local_DA_03": getData(cw_localDA_03).stack(),
    "local_DA_04": getData(cw_localDA_04).stack(),
    "local_DA_05": getData(cw_localDA_05).stack(),
    "local_DA_09": getData(cw_localDA_09).stack(),
    "avg_DA": getData(cw_avgDA).stack(),
    "chain_DA": getData(cw_chainDA).stack()
})
data.index.names=["height", "chain", "metric"]

Store the data so that we don't have to recompute it each time we run the notebook

In [ ]:
data.to_pickle("DA-data.pkl.gz", protocol=5, compression='gzip')